In [ ]:
import json
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime

load_dotenv(override=True)

In [ ]:
server_params = {"command": "python", "args": ["mcp-servers/assertion-server/assertion_server_python.py"]}

async with MCPServerStdio(params=server_params, client_session_timeout_seconds=60) as assertionServer:
    assertion_tools = await assertionServer.list_tools()
    result1 = await assertionServer.call_tool("assert_equals", {
            "actual": "hello",
            "expected": "helloo"
        })
   
print(assertion_tools)
data = json.loads(result1.content[0].text)
print(data)
passes_value = data["passed"]
passes_value

In [ ]:
# ok, now try different way with session
async def test_assertion_server():
    # Connect to the assertion server
    server_params = StdioServerParameters(
        command="python",
        args=["mcp-servers/assertion-server/assertion_server_python.py"]
    )
    
    # Use async context manager (async with, not await)
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the session
            await session.initialize()
            
            # List available tools
            print("📋 Available Tools:")
            tools_response = await session.list_tools()
            for tool in tools_response.tools:
                print(f"  - {tool.name}: {tool.description}")
            
            print("\n🧪 Testing Assertions:")
            
            # Test 1: assert_equals (should pass)
            result1 = await session.call_tool("assert_equals", {
                "actual": "hello",
                "expected": "hello"
            })
            print(f"\n1. assert_equals: {result1.content[0].text}")
            
            # Test 2: assert_equals (should fail)
            result2 = await session.call_tool("assert_equals", {
                "actual": "hello",
                "expected": "world"
            })
            print(f"\n2. assert_equals (fail): {result2.content[0].text}")
            
            # Test 3: assert_contains (should pass)
            result3 = await session.call_tool("assert_contains", {
                "text": "Hello World",
                "substring": "World"
            })
            print(f"\n3. assert_contains: {result3.content[0].text}")
            
            # Test 4: assert_count
            result4 = await session.call_tool("assert_count", {
                "actual": 5,
                "expected": 5
            })
            print(f"\n4. assert_count: {result4.content[0].text}")
            
            # Test 5: assert_greater_than
            result5 = await session.call_tool("assert_greater_than", {
                "actual": 10,
                "expected": 5
            })
            print(f"\n5. assert_greater_than: {result5.content[0].text}")



In [ ]:
await test_assertion_server()